In [2]:
# Load texts DRIVE
import numpy
texts = numpy.load("../2. Modelos de Lenguaje (Ring)/CohereFiles/texts.npy")
adapted_texts = numpy.load("../2. Modelos de Lenguaje (Ring)/CohereFiles/adapted_texts.npy")
levels_destined = numpy.load("../2. Modelos de Lenguaje (Ring)/CohereFiles/levels_destined.npy")
adapted_texts_levels = numpy.load("../2. Modelos de Lenguaje (Ring)/CohereFiles/adapted_texts_levels.npy")

In [3]:
# Filter Precisicion
n = len(texts)
level_to_number = {
    "A1": 0,
    "A2": 1,
    "B1": 2,
    "B2": 3,
    "C1": 4,
    "C2": 5
}

f_texts = []
f_adapted_texts = []
f_levels_destined = []
f_adapted_texts_levels = []

for i in range(n):
  level_diff = abs(level_to_number[levels_destined[i]] - level_to_number[adapted_texts_levels[i]])
  if(level_diff == 0):
    f_texts.append(texts[i])
    f_adapted_texts.append(adapted_texts[i])
    f_levels_destined.append(levels_destined[i])
    f_adapted_texts_levels.append(adapted_texts_levels[i])

In [4]:
# Calculate percentaje of accuracy
n = len(f_texts)
level_to_number = {
    "A1": 0,
    "A2": 1,
    "B1": 2,
    "B2": 3,
    "C1": 4,
    "C2": 5
}

counter_exact = 0
counter_aprox = 0
for i in range(n):
  level_diff = abs(level_to_number[f_levels_destined[i]] - level_to_number[f_adapted_texts_levels[i]])
  if(level_diff == 0):
    counter_exact += 1
    counter_aprox += 1
  elif(level_diff == 1):
    counter_aprox += 0.5

percentage_exact = counter_exact/n
percentage_aprox = counter_aprox/n
print("Porcentaje de precisión (Exacto): ", percentage_exact)
print("Porcentaje de precisión (Aproximado): ", percentage_aprox)
print("Cantidad de Textos: ", n)

Porcentaje de precisión (Exacto):  1.0
Porcentaje de precisión (Aproximado):  1.0
Cantidad de Textos:  372


In [5]:
# Hacerlo Dataframe
import pandas as pd

# Crear un DataFrame con los arreglos
df = pd.DataFrame({
    'text': f_adapted_texts,
    'label': f_adapted_texts_levels,
})

# Cantidad de cada nivel:
df.label.value_counts(dropna=False)

,count
label,
A1,116
A2,85
B2,50
C1,45
B1,41
C2,35


In [6]:
prompt = lambda label, text:  f"""

A continuación, te proporcionaré un texto en español y te pediré que lo modifiques para diferentes niveles de competencia lingüística
(A1, A2, B1, B2, C1 y C2), concretamente: {label}. El objetivo es que adaptes el texto según el nivel de dificultad, modificando el
vocabulario y las estructuras gramaticales para que se ajusten a cada nivel, pero manteniendo el mismo mensaje central. Solo responde
con la version del texto modificada para dicho nivel. No incluyas ninguna introducción, título, explicación o comentario. Solamente dame
el texto adaptado.

Aquí está el texto:
{text}
"""

In [7]:
prompts = []
for i in range(len(f_texts)):
  prompts.append(prompt(f_levels_destined[i], f_texts[i]))

In [8]:
# Crear y guardar el archivo JSONL
import json

datatemp = []

# Construcción del dataset en formato JSONL
for prompt, response in zip(prompts, adapted_texts):
  data_entry = {
      "messages": [
          {"role": "User", "content": prompt},
          {"role": "Chatbot", "content": response}]
      }
  datatemp.append(data_entry)

with open("/Datasets/exacto.jsonl", "w") as f:
  for entry in datatemp:
     f.write(json.dumps(entry) + "\n")